In [1]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import pandas as pd
from binary_c_API import evolve_binary
import os
import rebound
from rebound.interruptible_pool import InterruptiblePool
import numba

mpl.rc('font',**{'family':'serif','serif':['Palatino']})
# mpl.rc('text', usetex=True)
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['axes.titlesize'] = 20

%matplotlib inline

# Better looking figures
%config InlineBackend.figure_format = 'retina'

# Make cells narrower for better typography
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:65% !important; }</style>"))

In [2]:
def simulation(params):
    """
    Runs a single rebound simulation and calculates the MEGNO value.
    
    This function sets up a single rebound simulation, and calculates the MEGNO value
    based on the approach described in the Rein & Tamayo paper on variational equations.
    
    Parameters
    -----------
    m1 : float
        Mass of the first body in msun.
    m2 : float
        Mass of the second body in msun.
    m3 : float
        Mass of the third body in msun.
    P_i : float
        Period of the 'inner binary' in days.
    e_i : float
        Eccentricity of the inner binary.
    P_o : float
        Period of the 'outer binary' in days.
    e_o : float
        Eccentricity of the outer binary
    
    Returns
    -------
    megno : float
        The MEGNO value for the chosen system parameters.
    """

    m1, m2, m3, P_i, e_i, P_o, e_o = params
    
    sim = rebound.Simulation()
    sim.integrator = "whfast"
    sim.ri_whfast.safe_mode = 0
    
    # Convert periods to rebound units
    P_i *= 2*np.pi/365.25 # 2pi*years
    P_o *= 2*np.pi/365.25 
    
    sim.dt = P_i/10
    sim.add(m=m1)
    sim.add(m=m2, P=P_i, e=e_i) 
    sim.add(m=m3, P=P_o, e=e_o) 

    sim.move_to_com()
    sim.init_megno()
    sim.exit_max_distance = 20.
    try:
        sim.integrate(5e2*2.*np.pi, exact_finish_time=0) # integrate for 500 years, integrating to the nearest
        #timestep for each output to keep the timestep constant and preserve WHFast's symplectic nature
        megno = sim.calculate_megno()
        return megno
    except rebound.Escape:
        return 10. # At least one particle got ejected, returning large MEGNO.

@numba.jit
def resonance_width(m1, m2, m3, e_i, e_o, n):
    """
    Calculates the resonance width of an n-th order resonance in units of the 
    'inner binary' period. 
    """
    m123 = m1 + m2 + m3
    m12 = m1 + m2
    
    xi = np.arccosh(1/e_o) - np.sqrt(1 - e_o**2)
    width = (6*0.71**.5/((2*np.pi)**(1/4.)))*\
            (m3/m123 + n**(2/3.) *(m12/m123)**(2/3.)*(m1*m2/m12**2))**.5 *\
            (e_i**.5/e_o)*(1 - 13/24.*e_i**2.)**.5*(1 - e_o**2)**(3/8.)*\
            n**(3/4.)*np.exp(-n*xi/2.);
    return width

@numba.jit
def stability_plot(ax, params):
    """
    Plots a MEGNO map of a given system, together with resonant widths.
    
    For a given set of system parameters, the function plots a heat map 
    of MEGNO values as a function of period and 'outer binary' eccentricity.
    On top of this map it also plots the analytical prediction for resonant widths.
    
    Parameters
    ----------
    ax : matplotlib axes object
    
    m1 : float
        Mass of the first body in msun.
    m2 : float
        Mass of the second body in msun.
    m3 : float
        Mass of the third body in msun.
    P_i : float
        Period of the 'inner binary' in days.
    e_i : float
        Eccentricity of the inner binary."""

    m1, m2, m3, P_i, e_i = params

    # Grid resolution
    N = 120 
    
    par_P = np.linspace(3., 15., N)*P_i
    par_e = np.linspace(0., 0.8, N)

    parameters = []

    for e in par_e:
        for P in par_P:
            parameters.append((m1, m2, m3, P_i, e_i, P, e))

    # This part uses parallelization 
    pool = InterruptiblePool()
    results = pool.map(simulation, parameters)
    results2d = np.array(results).reshape(N, N)
    
    # Plot MEGNO stability map
    x = (par_P - (par_P[1] - par_P[0])/2)/P_i # shift ticks to center of cell
    y = par_e - (par_e[1] - par_e[0])/2
    im = ax.pcolormesh(x, y, results2d, vmin=1.9, vmax=4)
    
    im.set_edgecolor('face') # remove white lines in some pdf viewers
    ax.set_xlim(min(par_P/P_i), max(par_P/P_i))
    ax.set_xlabel(r"period ratio $P_o/P_i$")
    ax.set_ylim(min(par_e), max(par_e))
    ax.set_ylabel(r"eccentricity $e$")
    ax.set_xticks(np.arange(3, 16))
    ax.grid(True)
    
    cb = plt.colorbar(im, ax=ax)
    cb.set_label(r"MEGNO $\langle Y \rangle$")
    
    # Plot resonance widths
    e_o = np.linspace(0, 1, 500) 
    
    for i, n in enumerate(range(3, 16)):
        width = resonance_width(m1, m2, m3, e_i, e_o, n)   
        ax.fill_betweenx(e_o, -width + n, width + n, facecolor='black', alpha=0.15)

In [3]:
M1 = np.array([1.6, 2.0])
q = np.array([0.3, 0.6, 0.8])
P = np.array([20, 200])
e = np.array([0., 0.1, 0.3])

# from itertools import product
# for M1_, q_, P_, e_ in product(M1, q, P, e):
#     directory = "nbody_sims/{:.1f}_{:.1f}_{:d}_{:.1f}/".format(M1_, q_, int(P_), e_)
#     m_planet = 1e-03
#     params = [M1_, q_*M1_, m_planet, P_, e_]
#     fig, ax = plt.subplots(figsize=(10, 8))
#     stability_plot(ax, params)
    
    # Save to pdf
#     plt.savefig(directory + "m3_{:.4f}.pdf".format(m_planet))